In [1]:
import sys 
sys.path.append(r"C:\Users\a-sochat\development\Qcodes_Sohail_Clone\Qcodes")

import qcodes as qc
from qcodes.instrument.parameter import ManualParameter
from qcodes.sweep.sweep import Nest, Zip, Chain, ParameterSweep, ParameterWrapper, FunctionSweep, FunctionWrapper

In [125]:
def pretty_print(dictionary): 
    
    s = ""
    ss = ""
    
    names = list(dictionary.keys())[::-1]
    
    for name in names: 
        if dictionary[name]["independent_parameter"]: 
            s += ss + name + str(dictionary[name]["value"])
        else: 
            s += ss + name + "({})".format(s)
        
        ss = ", "
    
    print(s)

# Lets introduce a basic sweep object 

In [126]:
x = ManualParameter("x")

In [127]:
sox = ParameterSweep(x, lambda: [0, 1])

In [128]:
for i in sox: 
    pretty_print(i)

x0
x1


We have generated a small 1D coordinate layout with size 2 

# How do we make a 2D sweep? 

In [129]:
y = ManualParameter("y")

In [130]:
sox = ParameterSweep(x, lambda: [0, 1])
soy = ParameterSweep(y, lambda: [0, 1])

In [131]:
so_nest = Nest([soy, sox])  # X is the inner axis 

In [132]:
for i in so_nest: 
    pretty_print(i)

y0, x0
y0, x1
y1, x0
y1, x1


This represents a 2D layout of 2x2 

# We can extend this to ND

In [133]:
z = ManualParameter("z")

In [134]:
sox = ParameterSweep(x, lambda: [0, 1])
soy = ParameterSweep(y, lambda: [0, 1])
soz = ParameterSweep(z, lambda: [0, 1])

In [135]:
so_nest = Nest([soz, soy, sox])

In [136]:
for i in so_nest: 
    pretty_print(i)

z0, y0, x0
z0, y0, x1
z0, y1, x0
z0, y1, x1
z1, y0, x0
z1, y0, x1
z1, y1, x0
z1, y1, x1


We have created a 2x2x2 layout 

# This is how we can perform a measurement  

In [137]:
m = ManualParameter("m")
measurement = ParameterWrapper(m)

In [138]:
sox = ParameterSweep(x, lambda: [0, 1])
soy = ParameterSweep(y, lambda: [0, 1])

so_nest = Nest([soy, sox, measurement]) # Perform a measurement at every (x, y)

In [139]:
for i in so_nest: 
    pretty_print(i)

y0, x0, m(y0, x0)
y0, x1, m(y0, x1)
y1, x0, m(y1, x0)
y1, x1, m(y1, x1)


The meaning of "m(x0, y0)": We measure "m" at (x0, y0). We have thus a 2x2 coordinate layout and the thrid dimension represents a measurement 

# What happens if we nest a sweep in a measurement?

In [140]:
so_nest = Nest([measurement, sox])

In [142]:
for i in so_nest: 
    pretty_print(i)

m(), x0
m(), x1


Here "m()" represents the same measurement. We thus only call "m" once but the result will be echoed in all subsequent lines in the measurement table 

# Introducing chaining

In [151]:
m = ManualParameter("m")
mm = ParameterWrapper(m)

n = ManualParameter("n")
nn = ParameterWrapper(n)

In [152]:
sox = ParameterSweep(x, lambda: [0, 1])

so_chain = Chain([mm, Nest([sox, nn])])  # Perform measurement1 and then iterate over the sweep
# This is approx. equivalent to m + (x * n) 

In [153]:
for i in so_chain: 
    pretty_print(i)

m()
x0, n(x0)
x1, n(x1)


"m()" means that we measure m at undefined x or y 

In [154]:
sox = ParameterSweep(x, lambda: [0, 1])
soy = ParameterSweep(y, lambda: [0, 1])

so_chain = Nest([soy, Chain([mm, Nest([sox, nn])])])  # = y * (m + x * n) = ym + yxn

In [155]:
for i in so_chain: 
    pretty_print(i)

y0, m(y0)
y0, x0, n(y0, x0)
y0, x1, n(y0, x1)
y1, m(y1)
y1, x0, n(y1, x0)
y1, x1, n(y1, x1)


We have woven together a 1D and 2D loop

# We will introduce a couple of conviniece functions so we do not have to be as verbose

In [148]:
from qcodes.sweep import sweep, nest, chain

In [149]:
sox = sweep(x, [0, 1])
soy = sweep(y, [0, 1])

m = ManualParameter("m")
n = ManualParameter("n")

so_chain = nest(soy, chain(m, nest(sox, n)))

In [150]:
for i in so_chain: 
    pretty_print(i)

y0, m(y0)
y0, x0, n(y0, x0)
y0, x1, n(y0, x1)
y1, m(y1)
y1, x0, n(y1, x0)
y1, x1, n(y1, x1)
